<a href="https://colab.research.google.com/github/sid-betalol/Document-Classification-Models/blob/main/create_rvlcdip_subset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] -U
!pip install datasets

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import AutoFeatureExtractor
from PIL import Image
import io
from tqdm.auto import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
torch.manual_seed(42)

In [5]:
task = 'rvl-cdip'
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/dit-base")
num_labels = 16
samples_per_class_train = 100
samples_per_class_eval = 10

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/302 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/beit/feature_extraction_beit.py:28: FutureWarning: The class BeitFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use BeitImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/beit/image_processing_beit.py:110: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [6]:
def preprocess_and_save(dataset_name, split, feature_extractor, samples_per_class, num_labels, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    # Adjusted to include trust_remote_code=True based on the warning
    dataset_stream = load_dataset(dataset_name, split=split, streaming=True, trust_remote_code=True)
    samples_collected = {label: 0 for label in range(num_labels)}

    for example in tqdm(dataset_stream):
        label = example['label']
        if samples_collected[label] < samples_per_class:
            samples_collected[label] += 1
            # Check if the image is a bytes object or already an image object
            if isinstance(example['image'], bytes):
                # If it's bytes, convert it to an image
                image = Image.open(io.BytesIO(example['image']))
            else:
                # If it's already an image, use it directly
                image = example['image']

            if image.mode != 'RGB':
                image = image.convert('RGB')
            pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values.squeeze()
            torch.save((pixel_values, label), os.path.join(save_dir, f"{split}_{label}_{samples_collected[label]}.pt"))

        if all(value == samples_per_class for value in samples_collected.values()):
            break

In [7]:
preprocess_and_save("rvl_cdip", 'train', feature_extractor, samples_per_class_train, num_labels, f"/content/drive/MyDrive/rvl-cdip-pp/DiT/{task}/train_data")
preprocess_and_save("rvl_cdip", 'test', feature_extractor, samples_per_class_eval, num_labels, f"/content/drive/MyDrive/rvl-cdip-pp/DiT/{task}/eval_data")

0it [00:00, ?it/s]

0it [00:00, ?it/s]